In [1]:
import os

# Move to you working path
os.chdir("D:\Programing\Projects\Computer Vision\Vehicle Detection and Counting Using OpenCV")

In [3]:
import cv2
import numpy as np

In [6]:
width_min = 80
height_min = 80
offset = 6
ypos_line = 550

detec = []

def contour_center(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy

In [14]:
# video source input
cap = cv2.VideoCapture('test.mp4')
result = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (1280,720))

BGS = cv2.bgsegm.createBackgroundSubtractorMOG()
vehicles = 0


while True:
    
    ret, frame = cap.read()
    
    if ret is not True:
        break
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 5)
    img_sub = BGS.apply(blur)
    dilate = cv2.dilate(img_sub, np.ones((5,5)))
    
    contours, _ = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.line(frame, (25, ypos_line), (1200, ypos_line), (176, 130, 39), 2)
    
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        
        # check if the width and height of the contour is larger than the min width and height
        # so the distant vehicles will not be detected
        if ((w >= width_min) and (h >= height_min)) is not True:
            continue
            
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        center = contour_center(x, y, w, h)
        detec.append(center)
        
        cv2.circle(frame, center, 4, (0, 0, 255), -1)
        
        for (x, y) in detec:                 
            # check if the contour center passes on the line
            if (y < (ypos_line + offset)) and (y > (ypos_line-offset)):
                vehicles += 1
                cv2.line(frame, (25, ypos_line), (1200, ypos_line), (0, 127, 255), 3)
                detec.remove((x, y))
        
    cv2.putText(frame, "VEHICLE COUNT : "+str(vehicles), (320, 70), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 4)
    
    cv2.imshow("Video Original", frame)
    cv2.imshow("Detectar ", dilate)
    
    result.write(frame)
    
    if cv2.waitKey(1) == 27:
        break   

        
cap.release()
result.release()        
cv2.destroyAllWindows()